In [2]:
# Data import and exploratory 

import pandas as pd
import numpy as np
from datetime import datetime

# Data visualization


import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Creeting a cluster

from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from numpy import set_printoptions

#from scipy import stats

# Split the data and work with unbalanced class
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Model Creation

from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

#Evaluate the models

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,precision_score, recall_score,accuracy_score,classification_report

# setting  Jupyter outputs

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.options.display.float_format= '{:.2f}'.format

In [38]:
# Import the dataset

clients = pd.read_csv('Clients.csv')
target = pd.read_csv('Clients-target.csv')
clients_mov1 = pd.read_csv('Clients-mov-201604.201710.csv')
clients_mov2 = pd.read_csv('Clients-mov-201711.201806.csv')

In [40]:
clients.head()

,cid,branch_amount_1,branch_amount_2,branch_amount_3,branch_amount_4,promo,days_clients,branches,promos_sent,promos_buys,prod_unit,flagret,buy_frecuency,state,county,mid,date_birth
0,55600018227,0.00,0.00,192.00,0.00,19,511,1,8,0,106.88,0,511.00,MN,550,1821001,01/28/72
1,55600021052,144.69,0.00,16.19,0.00,4,91,2,0,0,27.40,0,91.00,SC,290,284301,10/06/51
2,55600021210,0.00,141.29,138.60,0.00,15,657,2,5,1,12.74,1,82.12,CA,947,1679802,05/12/71
3,55600021232,0.00,0.00,380.18,0.00,24,384,3,7,2,35.49,0,96.00,AR,722,1726702,07/10/63
4,55600028433,0.00,623.24,230.23,0.00,18,694,2,11,3,24.35,1,99.14,MD,207,2109903,08/29/55


In [41]:
clients_mov1.head()

,cid,last_month_buy,amount,payment_type,channel,buy,class,coupons,items
0,55600021210,201709,279.89,0,0,8,10,0,13
1,55600021232,201707,203.20,1,1,2,1,0,3
2,55600028433,201707,642.47,0,0,6,8,0,22
3,55600029142,201705,116.00,0,0,1,1,0,2
4,55600029445,201706,129.48,0,0,1,3,0,3


In [42]:
clients_mov2.head()

,cid,month,amount,payment_type,channel,buy,class,coupons,items
0,55600037073,201711,83.00,0,0,1,1,1,1
1,55600125936,201711,489.93,0,0,1,4,0,7
2,55600193584,201711,269.00,1,0,1,4,1,9
3,55600210784,201711,182.50,0,0,1,4,0,8
4,55600237820,201711,48.74,1,0,1,3,1,4


In [43]:
target.head()

,cid,flag_included,flag_buy
0,55600018227,1,0
1,55600021052,1,0
2,55600021210,0,0
3,55600021232,0,0
4,55600028433,1,0


### Concatenate clients_mov1 and clients_mov2
#### Rename columns: month_last_buy = month

In [44]:

clients_mov1.rename(columns={'mes_ultima_compra':'mes'},inplace=True)

data = ([clients_mov1,clients_mov2])
clients_mov = pd.concat(data)
clients_mov.head()

,cid,last_month_buy,amount,payment_type,channel,buy,class,coupons,items,month
0,55600021210,201709.00,279.89,0,0,8,10,0,13,NaN
1,55600021232,201707.00,203.20,1,1,2,1,0,3,NaN
2,55600028433,201707.00,642.47,0,0,6,8,0,22,NaN
3,55600029142,201705.00,116.00,0,0,1,1,0,2,NaN
4,55600029445,201706.00,129.48,0,0,1,3,0,3,NaN


In [45]:
clients_mov.shape

(49917, 10)

### Merge clients and target

In [46]:
data = pd.merge(clients,target,how='left')
data.head()

,cid,branch_amount_1,branch_amount_2,branch_amount_3,branch_amount_4,promo,days_clients,branches,promos_sent,promos_buys,prod_unit,flagret,buy_frecuency,state,county,mid,date_birth,flag_included,flag_buy
0,55600018227,0.00,0.00,192.00,0.00,19,511,1,8,0,106.88,0,511.00,MN,550,1821001,01/28/72,1,0
1,55600021052,144.69,0.00,16.19,0.00,4,91,2,0,0,27.40,0,91.00,SC,290,284301,10/06/51,1,0
2,55600021210,0.00,141.29,138.60,0.00,15,657,2,5,1,12.74,1,82.12,CA,947,1679802,05/12/71,0,0
3,55600021232,0.00,0.00,380.18,0.00,24,384,3,7,2,35.49,0,96.00,AR,722,1726702,07/10/63,0,0
4,55600028433,0.00,623.24,230.23,0.00,18,694,2,11,3,24.35,1,99.14,MD,207,2109903,08/29/55,1,0


### Data cleaning and transformation

In [47]:
from Data_clean import nullValues, MedMedMaxMin, pivot_table, uniqueRate, Zerovalues, neg
from Data_clean import Analisis_crosstab_target_vs, catPlotCount, Boxplot

In [48]:
Zerovalues(clients)

,TotalRecord,#Zeros,%Zeros
branch_amount_1,21740,20665,95.00
branch_amount_2,21740,12104,56.00
branch_amount_4,21740,18856,87.00
promo,21740,827,4.00
promos_sent,21740,3499,16.00
promos_buys,21740,11710,54.00
flagret,21740,14394,66.00


In [6]:
nullValues(clients)

,NullRecord,TotalRecord,CompleteRecord,Complete %,Empy %
fecha_nacimiento,1806,21740,19934,91.69,8.31


In [49]:
uniqueRate(clients,0)

,UniqueRate,UniqueValue
cid,1.00,21740
mid,1.00,21740
branch_amount_3,0.52,11358
date_birth,0.50,10813
prod_unit,0.30,6506
branch_amount_2,0.29,6303
buy_frecuency,0.19,4147
branch_amount_4,0.09,1969
county,0.04,868
branch_amount_1,0.04,769


In [51]:
uniqueRate(clients_mov,0)

,UniqueRate,UniqueValue
amount,0.44,22108
cid,0.44,21741
items,0.00,135
buy,0.00,74
class,0.00,26
coupons,0.00,13
last_month_buy,0.00,11
month,0.00,8
payment_type,0.00,2
channel,0.00,2
